In [97]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [98]:
df = pd.read_csv('LaunchData.csv')

In [99]:
df = df.drop(columns=['RocketName'])
x = df.drop(columns=['AltitudeHeight', 'TimeAtAltitude', 'TotalFlightTime'])
y = df[['AltitudeHeight', 'TimeAtAltitude', 'TotalFlightTime']]

In [100]:
x = pd.get_dummies(x, columns=['NoseType', 'FinShape'], drop_first=True)

In [101]:
xTrain, xVal, yTrain, yVal = train_test_split(x, y, test_size=0.2, random_state=42)

In [102]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(xTrain.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3)  
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [103]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

xTrain = xTrain.astype(np.float32)
xVal = xVal.astype(np.float32)
yTrain = yTrain.astype(np.float32)
yVal = yVal.astype(np.float32)

history = model.fit(
    xTrain, yTrain,
    validation_data=(xVal, yVal),
    epochs=200,
    batch_size=16,
    callbacks=[es],
    verbose=2
)

Epoch 1/200
12/12 - 1s - loss: 932990538972766666752.0000 - mae: 1684590336.0000 - val_loss: 60406844852315246034944.0000 - val_mae: 20984768512.0000 - 606ms/epoch - 50ms/step
Epoch 2/200
12/12 - 0s - loss: 932988005697976270848.0000 - mae: 1684727168.0000 - val_loss: 60406808823518227070976.0000 - val_mae: 20984840192.0000 - 60ms/epoch - 5ms/step
Epoch 3/200
12/12 - 0s - loss: 932985402054441697280.0000 - mae: 1684847744.0000 - val_loss: 60406772794721208107008.0000 - val_mae: 20984944640.0000 - 56ms/epoch - 5ms/step
Epoch 4/200
12/12 - 0s - loss: 932983713204581433344.0000 - mae: 1685046528.0000 - val_loss: 60406727758724934402048.0000 - val_mae: 20985088000.0000 - 58ms/epoch - 5ms/step
Epoch 5/200
12/12 - 0s - loss: 932980476242349260800.0000 - mae: 1685176320.0000 - val_loss: 60406709744326424920064.0000 - val_mae: 20985159680.0000 - 59ms/epoch - 5ms/step
Epoch 6/200
12/12 - 0s - loss: 932978928129977352192.0000 - mae: 1685271680.0000 - val_loss: 60406682722728660697088.0000 - val_

In [104]:
model.save('flightPathModel')
import json
with open('flightPathModel/columns.json', 'w') as f:
    json.dump(list(x.columns), f)

INFO:tensorflow:Assets written to: flightPathModel\assets
